__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-22 09:43:28 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-22 09:43:28 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-22 09:43:31 | AdsListParser: downloader init
[INFO    ] 2022-09-22 09:43:31 | AvitoParser: init
[INFO    ] 2022-09-22 09:43:37 | AvitoParser: 78 pages for read
[INFO    ] 2022-09-22 09:43:37 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-22 09:43:41 | AdsListParser: read page 1
[INFO    ] 2022-09-22 09:43:44 | AdsListParser: read page 2
[INFO    ] 2022-09-22 09:43:48 | AdsListParser: read page 3
[INFO    ] 2022-09-22 09:43:51 | AdsListParser: read page 4
[INFO    ] 2022-09-22 09:43:55 | AdsListParser: read page 5
[INFO    ] 2022-09-22 09:44:00 | AdsListParser: read page 6
[INFO    ] 2022-09-22 09:44:04 | AdsListParser: read page 7
[INFO    ] 2022-09-22 09:44:09 | AdsListParser: read page 8
[INFO    ] 2022-09-22 09:44:14 | AdsListParser: read page 9
[INFO    ] 2022-09-22 09:44:18 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3867
3631


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
352,2580103074,"Объявление «3-к. квартира, 71 м², 7/9 эт.» 13 фотографий",7500000,,"пр-т Победы, 62","Продаётся трёхкомнатная квартира на улице проспект Победы, 62. \nОбщая площадь квартиры 71 м², жилая — 48 кв. М. И кухня — 8 кв. М. \nВ квартире две лоджии. Комнаты все изолированы. Квартира расположена на седьмом этаже девятиэтажного дома. \nКвартира светлая",8,2022-09-22 09:48:34.368190,sevastopol
750,2524469039,"Объявление «Квартира-студия, 24,1 м², 2/3 эт.» 2 фотографии",1699000,,"Волнистая ул., 30",Продаю студию в самом лучшем районе города! \nПрописка! Материнский капитал! \nСтудия будет сдана в предчистовой отделке! \nРядом вся необходимая инфраструктура! \nВсе коммуникации городские! \nЗвоните в любой день.,16,2022-09-22 09:48:34.368190,sevastopol
